$\textbf{Random Number Generator based on Boson Sampling}$

Using all the information in the latest notebooks here we present a Random Quantum Number Generator.

Let's consider the problem in which we need a random string S of length L composed by $0's$ and $1's$. 
This is the scenario for some cryptography protocols. 

We are presenting a random number, based on the simulation of fock Boson Sampling.


$\textbf{Simulating the Boson Sampling for RN generation }$

In [15]:
from scipy.stats import unitary_group
import numpy as np
import strawberryfields as sf
from strawberryfields.ops import *
from random import randrange

# creating a Haar random unitary Matrix using scipy
#--------------------------------------
np.random.seed(42)

# define the random unitary for the linear interferometer
U = unitary_group.rvs(5)

#We're gonna use the same interferometer for the QRNG based on BS and based on GBS.



$\textbf{Random number generator using Fock Boson Sampling simulator}$

In [102]:
# States preparation
#--------------------------------------

# initialize a 5 mode program
boson_sampling = sf.Program(5)

# prepare the state
# 2 first modes 1 photon state
# Other modes in vaccuumù
with boson_sampling.context as q:
    for i in range (5):
        if i<3:
            Fock(1) | q[i]
        else :
            Vac     | q[i]
            
#---------------------------------------------------------------

    # apply the unitary transformation
    Interferometer(U) | q
    
#----------------------------------------------------------
    # Measure the ouput state
    MeasureFock() | q

# Measurement results
# -----------------------------

#Definition of the post-process.
def post_process(S1,S2):
    S=[] # create an empty list
    # fill that list by comparing bit by bit
    for i in range (len(S1)):
        if S1[i]!=0 and S2[i]==0:
            S = S + ['0']
        if S1[i]==0 and S2[i]!=0:
            S = S + ['1']
    Sample = np.array(S) # turn list in to array
    return S


#Function that generates the Fock random number
#-------------------------------------------
def Rstr_fock(L):
    rn=''
    eng = sf.Engine(backend="fock", backend_options={"cutoff_dim":3})
    sampl=[]
    i=0
    j=0
    while j<L:
        i+=1
        a=eng.run(boson_sampling)
        b=np.ravel(a.samples)
        sampl.append(b)
        #print('i=',i)
        if i>2:
            s1=sampl[randrange(i-1)]
            s2=sampl[randrange(i-1)]
            RN =post_process(s1,s2)
            if RN!=[]:   
                rn+=''.join(RN)
                j=len(rn)
    return rn



So in order to generate a random string of length L you just need to run the code 

$\textit{RN=Rstr_fock(L)}$

$\textit{print(RN[0:L])}$


In [104]:
#Fock random number generator


L=30
RN=Rstr_fock(L)
print(RN[0:L])

100110111010101101101010110100


This code is not in the final version since we spent most time in the understanding, the generation of random numbers is ricght now slow, but since we can garantee the unbiased distribution it's only a matter of time to improve the code, also if we have acces to a real hardware we could expect better times that in the simulation of BS. 